<a href="https://colab.research.google.com/github/shuwang127/MLClassifiers/blob/main/TestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

_COLAB_ = 0 if (os.getenv('COLAB_GPU', 'NONE') == 'NONE') else 1
rootPath = './drive/My Drive/Colab Notebooks/' if (_COLAB_) else './'

# read data.
traindata = pd.read_csv(rootPath + 'training_data.csv')
testdata = pd.read_csv(rootPath + 'testing_data_labels.csv')
# fill missing features.
traindata = traindata.replace(' ?', np.nan)
for index, row in traindata.iteritems():
    traindata[index].fillna(traindata[index].mode()[0], inplace=True)
testdata = testdata.replace(' ?', np.nan)
for index, row in testdata.iteritems():
    testdata[index].fillna(testdata[index].mode()[0], inplace=True)

In [2]:
# get labels.
YTrain = traindata['salary'].replace([' <=50K' , ' >50K'], [0, 1])
YTest = testdata['salary'].replace([' <=50K' , ' >50K'], [0, 1]) ##

# get features.
XTrain = traindata.drop(['ID (this is not a feature)', 'salary'], axis=1)
XTest = testdata.drop(['ID (this is not a feature)', 'salary'], axis=1) ##
# normalize numeric features.
from sklearn.preprocessing import StandardScaler
numcol = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()
XTrain[numcol] = scaler.fit_transform(XTrain[numcol])
XTest[numcol] = scaler.transform(XTest[numcol])
# get one-hot vectors.
X = pd.concat([XTrain,XTest])
X = pd.get_dummies(X)
XTrain = X[:len(XTrain)]
XTest = X[len(XTrain):]
print("size of training data is", XTrain.shape)
print("size of testing data is", XTest.shape)

size of training data is (38842, 105)
size of testing data is (10000, 105)


In [3]:
# evaluation.
from sklearn.metrics import accuracy_score, confusion_matrix
def Evaluation(YTest, YPred, method=''):
    print('====================== ' + method + ' ======================')
    acc = accuracy_score(YTest, YPred) * 100
    print('Accuracy is %.3f%%.' % acc)
    conf = confusion_matrix(YTest, YPred)
    print(pd.DataFrame(conf, columns=['Pred-Neg','Pred-Pos'], index=['Actl-Neg', 'Actl-Pos']))
    precision = conf[1][1] / (conf[0][1] + conf[1][1]) if (conf[0][1] + conf[1][1]) else 0
    recall = conf[1][1] / (conf[1][0] + conf[1][1]) if (conf[1][0] + conf[1][1]) else 0
    F1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    print('Precision is %.3f.' % precision)
    print('Recall is %.3f.' % recall)
    print('F1 score is %.3f.' % F1)

In [4]:
# Random Forest.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, max_depth=None, criterion='gini', max_features=None, min_samples_split=320, min_samples_leaf=4, n_jobs=-1, verbose=1, random_state=0)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Random Forest') ##

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  4.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.9s


====================== Random Forest ======================
Accuracy is 86.490%.
          Pred-Neg  Pred-Pos
Actl-Neg      7210       429
Actl-Pos       922      1439
Precision is 0.770.
Recall is 0.609.
F1 score is 0.681.


[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    1.1s finished


In [5]:
testdata['pred-salary'] = YPred
testdata['pred-salary'] = testdata['pred-salary'].replace([0, 1], [' <=50K' , ' >50K'])
result = testdata[['ID (this is not a feature)', 'pred-salary']]
result.to_csv(rootPath + 'result.csv', index=0)
result.head()

,ID (this is not a feature),pred-salary
0,1,<=50K
1,2,<=50K
2,3,>50K
3,4,>50K
4,5,<=50K
